In [1]:
# Perform imports
import polars as pl
from datetime import datetime
import altair as alt

from polars_backtesting_engine.polars_backtesting_engine.data_handler import DataHandler

In [2]:
# Get data
# exchange_code = "XASX"
# ticker = "BKW"
exchange_code = "XNAS"
ticker = "AAPL"
start = datetime(2024, 1, 1)
end = datetime.now()

data = DataHandler(
    exchange_code=exchange_code, ticker=ticker, start=start, end=end).load_data_from_local_database()
data = data.sort('date')

In [3]:
# Transform data
data = data.with_columns(# exponential moving average
                        pl.col("close").ewm_mean(span=5, adjust=False).alias('ema_5'),
                        pl.col("close").shift(1).ewm_mean(span=5, adjust=False).alias('prior_ema_5'),
                        pl.col("close").ewm_mean(span=20, adjust=False).alias('ema_20'),
                        pl.col("close").shift(1).ewm_mean(span=20, adjust=False).alias('prior_ema_20'),
                        pl.col("close").ewm_mean(span=50, adjust=True, ignore_nulls=True).alias('ema_50'),
                        pl.col("close").shift(1).ewm_mean(span=50, adjust=True, ignore_nulls=True).alias('prior_ema_50'),
                        pl.col("close").ewm_mean(span=200, adjust=True, ignore_nulls=True).alias('ema_200'),
                        pl.col("close").shift(1).ewm_mean(span=200, adjust=True, ignore_nulls=True).alias('prior_ema_200'),
                        # Simple moving avaerage
                        pl.col("close").rolling_mean(10).alias("close_MA10"))
data = data.with_columns(pl.when((pl.col("ema_5") > pl.col("ema_20")) & (pl.col("prior_ema_5") <= pl.col("prior_ema_20")))
                            .then(pl.col("close")*0.98).alias("sig_bull_5_20"),
                        pl.when((pl.col("ema_5") < pl.col("ema_20")) & (pl.col("prior_ema_5") >= pl.col("prior_ema_20")))
                            .then(pl.col("close")*0.98).alias("sig_bear_5_20"),
                        pl.when((pl.col("ema_20") > pl.col("ema_50")) & (pl.col("prior_ema_20") <= pl.col("prior_ema_50")))
                            .then(pl.col("close")*0.98).alias("sig_bull_20_50"),
                        pl.when((pl.col("ema_20") < pl.col("ema_50")) & (pl.col("prior_ema_20") >= pl.col("prior_ema_50")))
                            .then(pl.col("close")*0.98).alias("sig_bear_20_50"),
                        pl.when((pl.col("ema_50") > pl.col("ema_200")) & (pl.col("prior_ema_50") <= pl.col("prior_ema_200")))
                            .then(pl.col("close")*0.98).alias("sig_bull_50_200"),
                        pl.when((pl.col("ema_50") < pl.col("ema_200")) & (pl.col("prior_ema_50") >= pl.col("prior_ema_200")))
                            .then(pl.col("close")*0.98).alias("sig_bear_50_200"),)

In [4]:
# Plot data using a long dataframe format for price data

# First convert from wide to long using unpivot
columns = ['date', 'close', 'ema_50', 'ema_200']
line_columns = data.select(columns)
long_data = line_columns.unpivot(index='date', on=['close', 'ema_50', 'ema_200'], variable_name='type', value_name='value')

line_chart = (
    alt.Chart(long_data)
    .mark_line(strokeWidth=1)
    .encode(x=alt.X("date:T",
                    title='',
                    axis=alt.Axis(labelAngle=-45, tickColor="#09080B", domainColor="#09080B", grid=False, labelFontSize=10, titleFontSize=12)),
            y=alt.Y("value:Q",
                    title='Close',
                    scale=alt.Scale(zero=False),
                    axis=alt.Axis(format='$.2f', tickColor="#09080B", domainColor="#09080B", gridColor="#9DCDF8", grid=True, labelFontSize=10, titleFontSize=12)
                    ),
            color="type:N",
            tooltip=['date:T', 'value:Q'])
    ).properties(title={'text': 'Stock Price Analysis',
                        'subtitle': ['50 and 200-day exponential moving average overlay'],
                        'fontSize': 16
                },
                width=500,
                height=300,
    )

xrule = (
        alt.Chart()
        .mark_rule(color="#00FFFF", strokeWidth=2)
        .encode(x=alt.datum(alt.DateTime(year=2024, month="November")))
)

yrule = (
    alt.Chart().mark_rule(color="#1010AA", strokeDash=[12, 6], size=2).encode(y=alt.datum(220))
)

bull_sig = (
    alt.Chart(data)
    .mark_point(shape='triangle-up',color="#0F9A0F",size=30, strokeWidth=3)
    .encode(x='date:T',
            y='sig_bull_50_200'))

bear_sig = (
    alt.Chart(data)
    .mark_point(shape='triangle-down',color='#FF0000',size=30, strokeWidth=3)
    .encode(x='date:T',
            y='sig_bear_50_200'))

#Layer the above five together
line_chart = (line_chart + xrule + yrule + bull_sig + bear_sig).interactive()

bar_chart = alt.Chart(data).mark_bar(color="#4286B4",opacity=1.0
    ).encode(x=alt.X("date:T",
                title='Date',
                axis=alt.Axis(labelAngle=-45, tickColor="#09080B", domainColor="#09080B", grid=False, labelFontSize=10, titleFontSize=12)
                ),
            y=alt.Y('volume:Q',
                    title='Volume',
                    axis=alt.Axis(tickColor="#09080B", domainColor="#09080B", gridColor="#9DCDF8", grid=True, labelFontSize=10,titleFontSize=12)
                    ),
            tooltip=['date:T', 'Volume:Q'],
    ).properties(title='Volume',
                width=500,
                height=150,)

# Vertically combine the charts
combined_chart = alt.vconcat(line_chart, bar_chart).resolve_scale(x='shared').configure(background="#CBE1F6")
combined_chart.show()


alt.VConcatChart(...)